In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import recordlinkage

In [2]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

In [3]:
df_sba = pd.read_csv("./data/PPP Data 150k plus - NY.csv")
df_ue = pd.read_csv("./data/new-york-details.csv")
df_sba.head(1)
df_ue.head(1)

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
0,a $5-10 million,1186 BROADWAY RESTAURANT LLC,30 W 26th Street Floor 12,NEW YORK,NY,10010.0,721110.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,417.0,04/15/2020,BMO Harris Bank National Association,NY - 12


,name,url,city,cuisine,avgRating,numReviews,priceRange,latitude,longitude,telephone,postalCode,streetAddress,addressLocality,addressRegion,openingHoursSpecification,addressString,geoString,ratingString
0,Market Cafe,https://www.ubereats.com/new-york/food-deliver...,new-york,"['Pizza', 'American', 'Italian']",NaN,NaN,$,40.752349,-73.975021,1.212682e+10,10017,425 Lexington Ave,New York,NY,"[{'@type': 'OpeningHoursSpecification', 'dayOf...","{'@type': 'PostalAddress', 'addressLocality': ...","{'@type': 'GeoCoordinates', 'latitude': 40.752...",{}


## Pre-processing

In [4]:
# Converting Zipcodes in SBA data from `float` to `int`. Also there are some cases where the leading zeroes may have been lost (due to erronious conversion to `float`).

# | zipcode       | zipcode       |
# | ------------- |-------------  |
# | 11.0          | 00011         |
# | 11013.0       | 11013         |

df_sba['Zip'] = df_sba['Zip'].astype(int).astype(str).str.zfill(5)

In [5]:
# The names are all upper case in SBA data. So, doing the same for Uber Eats as well.
df_ue['name'] = df_ue['name'].str.upper()
df_ue['addressLocality'] = df_ue['addressLocality'].str.upper()
df_ue['addressRegion'] = df_ue['addressRegion'].str.upper()

In [6]:
# Taking out the useful columns
df_a = df_sba[['BusinessName', 'Address', 'City', 'State', 'Zip']]
df_b = df_ue[['name', 'streetAddress', 'addressLocality', 'addressRegion', 'postalCode']]

reqd_columns = ['name', 'street', 'city', 'state', 'zip']

df_a.columns = reqd_columns
df_b.columns = reqd_columns

In [7]:
df_a.head(1)
df_b.head(1)

,name,street,city,state,zip
0,1186 BROADWAY RESTAURANT LLC,30 W 26th Street Floor 12,NEW YORK,NY,10010


,name,street,city,state,zip
0,MARKET CAFE,425 Lexington Ave,NEW YORK,NY,10017


In [8]:
# Counting NaN values
df_a.apply(lambda x: x.isna().sum())
df_b.apply(lambda x: x.isna().sum())

name      2
street    0
city      0
state     0
zip       0
dtype: int64

name       23
street    261
city      261
state     261
zip       280
dtype: int64

In [9]:
df_a[df_a['name'].isnull()]
df_b[df_b['name'].isnull()]

,name,street,city,state,zip
1547,NaN,820 Washington Street,NEW YORK,NY,10014
1548,NaN,2348 Adam Clayton Powell Jr. Blvd,NEW YORK,NY,10030


,name,street,city,state,zip
284,NaN,NaN,NaN,NaN,NaN
466,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN
915,NaN,NaN,NaN,NaN,NaN
1149,NaN,NaN,NaN,NaN,NaN
1150,NaN,NaN,NaN,NaN,NaN
1443,NaN,NaN,NaN,NaN,NaN
1472,NaN,NaN,NaN,NaN,NaN
1479,NaN,NaN,NaN,NaN,NaN
1523,NaN,NaN,NaN,NaN,NaN


In [10]:
# Each entry in Uber Eats DF with `NaN` in `name` necessarily has `NaN` in all other columns. Thus, we will drop these rows.
# If the address street matches perfectly there is a chance that the match is true, indeed. Thus, we will simply replace the `name` with an empty string instead of dropping the rows.
df_b = df_b.dropna(subset=['name'])
df_a = df_a.fillna({'name':""})
df_b = df_b.fillna({'street':"", 'city':"", 'state':"", 'zip':""})

In [11]:
# Counting NaN values
df_a.apply(lambda x: x.isna().sum())
df_b.apply(lambda x: x.isna().sum())

name      0
street    0
city      0
state     0
zip       0
dtype: int64

name      0
street    0
city      0
state     0
zip       0
dtype: int64

In [12]:
# Checking the columns for inconsistencies
df_b['city'].value_counts()
df_b['state'].value_counts()
df_b['zip'].apply(lambda x: len(x)).value_counts()

NEW YORK                 2892
                          238
NEW YORK CITY              94
MANHATTAN                  16
NEW YORK,                   8
NY                          6
BRONX                       4
GRIMACY                     1
NY 10003                    1
LONDON                      1
NORTH CHARLESTON            1
WEST VILLAGE NEW YORK       1
NEW YORK, NY                1
NEW YOTK                    1
THE BRONX                   1
NOLITA                      1
Name: city, dtype: int64

NY                  2587
NEW YORK             430
                     238
801 8TH AVE            1
195 SPRING ST          1
93 AVENUE B            1
LONDON                 1
342 E 6TH ST           1
167 1ST AVE            1
462-466 2ND AVE,       1
16 W 8TH ST            1
689 10TH AVE           1
SC                     1
48 MACDOUGAL ST        1
11 W 32ND ST           1
Name: state, dtype: int64

5     2994
0      257
10      13
2        2
6        1
Name: zip, dtype: int64

In [13]:
df_b['city'] = df_b['city'].value_counts().keys()[0]
df_b['state'] = df_b['state'].value_counts().keys()[0]
df_b['zip'] = df_b['zip'].apply(lambda x: "" if len(x) != 5 else x)

In [14]:
# Verifying
df_b['city'].value_counts()
df_b['state'].value_counts()
df_b['zip'].apply(lambda x: len(x)).value_counts()

NEW YORK    3267
Name: city, dtype: int64

NY    3267
Name: state, dtype: int64

5    2994
0     273
Name: zip, dtype: int64

## Using Recordlinkage

In [15]:
indexer = recordlinkage.Index()
indexer.block('zip')

<Index>

In [16]:
candidates = indexer.index(df_a, df_b)
print(len(candidates))

1501243


In [17]:
compare = recordlinkage.Compare()
# compare.exact('city', 'Provider City', label='City')
compare.string('name',
            'name',
            method='jarowinkler',
            threshold=0.75,
            label='name_low')
compare.string('name',
            'name',
            method='jarowinkler',
            threshold=0.9,
            label='name_high')
compare.string('street',
            'street',
            method='jarowinkler',
            threshold=0.85,
            label='street')
# compare.exact('zip',
#             'zip',
#             label='zip')
features = compare.compute(candidates, df_a, df_b)
features.sum(axis=1).value_counts().sort_index(ascending=False)

<Compare>

<Compare>

<Compare>

3.0         14
2.0         53
1.0      10513
0.0    1490663
dtype: int64

In [21]:
potential_matches = features[features.sum(axis=1) > 1].reset_index()
potential_matches['score'] = potential_matches.loc[:, 'name_low':'street'].sum(axis=1)

## Data Wrangling